In [8]:
import pandas as pd
import numpy as np

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV

In [10]:
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Nadam

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

# Load your training dataset
train_df = pd.read_csv('new1_cyber_train.csv')
# Load your testing dataset
test_df = pd.read_csv('new1_cyber_test.csv')


In [12]:
# Data Preprocessing
# Find the maximum number of unique words in the dataset
max_words = 100

# Find the maximum length of messages in the dataset
max_len = 50


def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Apply stemming
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text


# Preprocessing for training data
train_df['text'] = train_df['text'].apply(preprocess_text)

# Split the training dataset
X_train, y_train = train_df['text'], train_df['label']

# Tokenization
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

# Sequence and Padding for training data
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post')

# Preprocessing for testing data
test_df['text'] = test_df['text'].apply(preprocess_text)

# Split the testing dataset
X_test, y_test = test_df['text'], test_df['label']

# Sequence and Padding for testing data
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post')




## Building and Training Models

## DNN

In [6]:
# dnn_model_modified = Sequential([
#     Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
#     Flatten(),
#     Dense(32, activation='sigmoid'),
#     Dropout(0.2),
#     Dense(16, activation='relu'),
#     Dropout(0.7),  # Increased dropout rate
#     Dense(1, activation='sigmoid')
# ])

def create_dnn_model(optimizer='adam',learning_rate=0.001):
    dnn_model = Sequential([
        Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
        Flatten(),
        Dense(32, activation='sigmoid'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dropout(0.7),  # Increased dropout rate
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    dnn_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return dnn_model

# Wrap the Keras model in a scikit-learn compatible classifier
dnn_classifier = KerasClassifier(build_fn=create_dnn_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid_dnn = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search_dnn = GridSearchCV(estimator=dnn_classifier, param_grid=param_grid_dnn, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result_dnn = grid_search_dnn.fit(X_train_padded, y_train)


C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




Epoch 1/30


19/19 - 1s - loss: 0.7778 - accuracy: 0.4942 - 1s/epoch - 61ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7257 - accuracy: 0.4925 - 27ms/epoch - 1ms/step
Epoch 3/30
19/19 - 0s - loss: 0.6951 - accuracy: 0.5341 - 28ms/epoch - 1ms/step
Epoch 4/30
19/19 - 0s - loss: 0.7020 - accuracy: 0.5008 - 25ms/epoch - 1ms/step
Epoch 5/30
19/19 - 0s - loss: 0.6991 - accuracy: 0.4925 - 24ms/epoch - 1ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6922 - accuracy: 0.5275 - 25ms/epoch - 1ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6954 - accuracy: 0.4792 - 27ms/epoch - 1ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6933 - accuracy: 0.5324 - 33ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6936 - accuracy: 0.5008 - 32ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6971 - accuracy: 0.4975 - 33ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6923 - accuracy: 0.5291 - 30ms/epoch - 2ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6973 - accuracy: 0.4942 - 30ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


19/19 - 1s - loss: 0.8287 - accuracy: 0.4925 - 876ms/epoch - 46ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7307 - accuracy: 0.5058 - 35ms/epoch - 2ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7082 - accuracy: 0.4725 - 31ms/epoch - 2ms/step
Epoch 4/30
19/19 - 0s - loss: 0.6968 - accuracy: 0.4992 - 34ms/epoch - 2ms/step
Epoch 5/30
19/19 - 0s - loss: 0.6908 - accuracy: 0.5258 - 35ms/epoch - 2ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6917 - accuracy: 0.5042 - 33ms/epoch - 2ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6955 - accuracy: 0.4825 - 33ms/epoch - 2ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6902 - accuracy: 0.5225 - 33ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6905 - accuracy: 0.5308 - 32ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6908 - accuracy: 0.5258 - 33ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6939 - accuracy: 0.4975 - 29ms/epoch - 2ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6896 - accuracy: 0.5125 - 31ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6895 - accura

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

19/19 - 1s - loss: 0.7451 - accuracy: 0.4809 - 784ms/epoch - 41ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7118 - accuracy: 0.5125 - 29ms/epoch - 2ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7094 - accuracy: 0.5008 - 28ms/epoch - 1ms/step
Epoch 4/30
19/19 - 0s - loss: 0.6926 - accuracy: 0.4958 - 29ms/epoch - 2ms/step
Epoch 5/30
19/19 - 0s - loss: 0.7022 - accuracy: 0.5025 - 29ms/epoch - 2ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6899 - accuracy: 0.5141 - 29ms/epoch - 2ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6889 - accuracy: 0.5358 - 28ms/epoch - 1ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6940 - accuracy: 0.5291 - 31ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6877 - accuracy: 0.5308 - 30ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6853 - accuracy: 0.5607 - 30ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6869 - accuracy: 0.5408 - 30ms/epoch - 2ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6905 - accuracy: 0.5308 - 29ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6874 - accura

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


19/19 - 1s - loss: 0.7599 - accuracy: 0.5233 - 772ms/epoch - 41ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7272 - accuracy: 0.4850 - 29ms/epoch - 2ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7123 - accuracy: 0.4767 - 30ms/epoch - 2ms/step
Epoch 4/30
19/19 - 0s - loss: 0.6964 - accuracy: 0.5266 - 35ms/epoch - 2ms/step
Epoch 5/30
19/19 - 0s - loss: 0.6962 - accuracy: 0.5050 - 29ms/epoch - 2ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6965 - accuracy: 0.5017 - 31ms/epoch - 2ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6979 - accuracy: 0.4784 - 29ms/epoch - 2ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6909 - accuracy: 0.5083 - 30ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6940 - accuracy: 0.4618 - 32ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6932 - accuracy: 0.5183 - 29ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6898 - accuracy: 0.5199 - 29ms/epoch - 2ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6984 - accuracy: 0.4884 - 30ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6925 - accura

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


29/29 - 1s - loss: 0.8094 - accuracy: 0.5044 - 769ms/epoch - 27ms/step
Epoch 2/30
29/29 - 0s - loss: 0.7247 - accuracy: 0.5089 - 41ms/epoch - 1ms/step
Epoch 3/30
29/29 - 0s - loss: 0.7084 - accuracy: 0.4701 - 43ms/epoch - 1ms/step
Epoch 4/30
29/29 - 0s - loss: 0.7008 - accuracy: 0.4834 - 42ms/epoch - 1ms/step
Epoch 5/30
29/29 - 0s - loss: 0.6963 - accuracy: 0.4800 - 42ms/epoch - 1ms/step
Epoch 6/30
29/29 - 0s - loss: 0.6969 - accuracy: 0.4867 - 41ms/epoch - 1ms/step
Epoch 7/30
29/29 - 0s - loss: 0.6964 - accuracy: 0.5000 - 43ms/epoch - 1ms/step
Epoch 8/30
29/29 - 0s - loss: 0.6951 - accuracy: 0.4945 - 50ms/epoch - 2ms/step
Epoch 9/30
29/29 - 0s - loss: 0.6911 - accuracy: 0.5200 - 52ms/epoch - 2ms/step
Epoch 10/30
29/29 - 0s - loss: 0.6910 - accuracy: 0.5100 - 51ms/epoch - 2ms/step
Epoch 11/30
29/29 - 0s - loss: 0.6930 - accuracy: 0.5000 - 45ms/epoch - 2ms/step
Epoch 12/30
29/29 - 0s - loss: 0.6865 - accuracy: 0.5266 - 43ms/epoch - 1ms/step
Epoch 13/30
29/29 - 0s - loss: 0.6737 - accura

In [7]:
# Display the results in a table
results_df_dnn = pd.DataFrame(grid_result_dnn.cv_results_)
results_df_dnn[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.743850,9
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.753865,5
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.764939,1
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.716216,14
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.666382,15
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.736109,10
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.763846,3
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.729439,12
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.752743,6
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.734969,11


In [8]:
# Print the best parameters and corresponding accuracy
print("Best Parameters (DNN): ", grid_result_dnn.best_params_)
print("Best Accuracy (DNN): ", grid_result_dnn.best_score_)

Best Parameters (DNN):  {'learning_rate': 0.01, 'optimizer': 'Adagrad'}
Best Accuracy (DNN):  0.7649390919158362


In [20]:
best_dnn_model = grid_result_dnn.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_dnn_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

8/8 - 0s - 49ms/epoch - 6ms/step
Test Accuracy: 0.8008849557522124
Precision: 0.75
Recall: 0.9026548672566371
F1-score: 0.8192771084337348


## LSTM

In [ ]:
# lstm_model = Sequential([
#     Embedding(input_dim=max_words, output_dim=32, input_length=max_len),
#     LSTM(64, return_sequences=True),  # Added return_sequences=True for stacking LSTM layers
#     Dropout(0.5),
#     LSTM(64),
#     Dropout(0.3),
#     Dense(32, activation='relu'),
#     Dropout(0.3),
#     Dense(16, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])


def create_lstm_model(optimizer=Adam, learning_rate=0.001):
    lstm_model = Sequential([
        Embedding(input_dim=max_words, output_dim=64, input_length=max_len),  # Increased output_dim for more expressive power
        LSTM(128, return_sequences=True),  # Increased LSTM units for more representational capacity
        Dropout(0.5),
        LSTM(128),  # Increased LSTM units for more representational capacity
        Dropout(0.5),  # Increased dropout rate for regularization
        Dense(64, activation='relu'),  # Increased number of neurons in Dense layers
        Dropout(0.3),  # Added dropout for regularization
        Dense(32, activation='relu'),  # Increased number of neurons in Dense layers
        Dropout(0.3),  # Added dropout for regularization
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return lstm_model

# Wrap the Keras model in a scikit-learn compatible classifier
lstm_classifier = KerasClassifier(build_fn=create_lstm_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid_lstm = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search_lstm = GridSearchCV(estimator=lstm_classifier, param_grid=param_grid_lstm, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result_lstm = grid_search_lstm.fit(X_train_padded, y_train)


In [ ]:
results_df_lstm = pd.DataFrame(grid_result_lstm.cv_results_)
results_df_lstm[['params', 'mean_test_score', 'rank_test_score']]

In [ ]:
print("Best Parameters (LSTM): ", grid_result_lstm.best_params_)
print("Best Accuracy (LSTM): ", grid_result_lstm.best_score_)

In [ ]:
best_lstm_model = grid_result_lstm.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_lstm_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


## Bi-LSTM

In [ ]:
def create_bi_lstm_model(optimizer=Adam, learning_rate=0.001, lstm_units=64, dropout_rate=0.3):
    bi_lstm_model = Sequential([
        Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
        Bidirectional(LSTM(units=lstm_units, return_sequences=True)),
        Bidirectional(LSTM(units=lstm_units)),
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    
    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'
    
    bi_lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return bi_lstm_model

# Wrap the Keras model in a scikit-learn compatible classifier
bi_lstm_classifier = KerasClassifier(build_fn=create_bi_lstm_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search = GridSearchCV(estimator=bi_lstm_classifier, param_grid=param_grid, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result = grid_search.fit(X_train_padded, y_train)



C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 12s - loss: 0.6938 - accuracy: 0.4958 - 12s/epoch - 646ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6923 - accuracy: 0.5424 - 1s/epoch - 78ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6898 - accuracy: 0.5308 - 1s/epoch - 76ms/step
Epoch 4/30
19/19 - 1s - loss: 0.6448 - accuracy: 0.6639 - 1s/epoch - 75ms/step
Epoch 5/30
19/19 - 1s - loss: 0.4966 - accuracy: 0.7787 - 1s/epoch - 74ms/step
Epoch 6/30
19/19 - 1s - loss: 0.4692 - accuracy: 0.7787 - 1s/epoch - 73ms/step
Epoch 7/30
19/19 - 1s - loss: 0.5028 - accuracy: 0.7820 - 1s/epoch - 74ms/step
Epoch 8/30
19/19 - 1s - loss: 0.4251 - accuracy: 0.8220 - 1s/epoch - 74ms/step
Epoch 9/30
19/19 - 1s - loss: 0.4228 - accuracy: 0.8120 - 1s/epoch - 74ms/step
Epoch 10/30
19/19 - 1s - loss: 0.4083 - accuracy: 0.8203 - 1s/epoch - 74ms/step
Epoch 11/30
19/19 - 1s - loss: 0.4031 - accuracy: 0.8203 - 1s/epoch - 75ms/step
Epoch 12/30
19/19 - 1s - loss: 0.3886 - accuracy: 0.8186 - 1s/epoch - 74ms/step
Epoch 13/30
19/19 - 1s - loss: 0.4009 - accura

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 14s - loss: 0.6942 - accuracy: 0.4992 - 14s/epoch - 749ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6913 - accuracy: 0.5208 - 1s/epoch - 56ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6828 - accuracy: 0.5923 - 1s/epoch - 55ms/step
Epoch 4/30
19/19 - 1s - loss: 0.5869 - accuracy: 0.7205 - 1s/epoch - 53ms/step
Epoch 5/30
19/19 - 1s - loss: 0.4863 - accuracy: 0.7854 - 1s/epoch - 55ms/step
Epoch 6/30
19/19 - 1s - loss: 0.4495 - accuracy: 0.8087 - 982ms/epoch - 52ms/step
Epoch 7/30
19/19 - 1s - loss: 0.4272 - accuracy: 0.8136 - 971ms/epoch - 51ms/step
Epoch 8/30
19/19 - 1s - loss: 0.4515 - accuracy: 0.8037 - 982ms/epoch - 52ms/step
Epoch 9/30
19/19 - 1s - loss: 0.4126 - accuracy: 0.8136 - 975ms/epoch - 51ms/step
Epoch 10/30
19/19 - 1s - loss: 0.4152 - accuracy: 0.8203 - 973ms/epoch - 51ms/step
Epoch 11/30
19/19 - 1s - loss: 0.4054 - accuracy: 0.8270 - 986ms/epoch - 52ms/step
Epoch 12/30
19/19 - 1s - loss: 0.3955 - accuracy: 0.8319 - 984ms/epoch - 52ms/step
Epoch 13/30
19/19 - 1s - 

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/30
19/19 - 13s - loss: 0.6937 - accuracy: 0.5008 - 13s/epoch - 679ms/step
Epoch 2/30
19/19 - 2s - loss: 0.6917 - accuracy: 0.5308 - 2s/epoch - 106ms/step
Epoch 3/30
19/19 - 2s - loss: 0.6772 - accuracy: 0.6306 - 2s/epoch - 104ms/step
Epoch 4/30
19/19 - 2s - loss: 0.5617 - accuracy: 0.7537 - 2s/epoch - 105ms/step
Epoch 5/30
19/19 - 2s - loss: 0.4887 - accuracy: 0.7970 - 2s/epoch - 109ms/step
Epoch 6/30
19/19 - 2s - loss: 0.4376 - accuracy: 0.8236 - 2s/epoch - 109ms/step
Epoch 7/30
19/19 - 2s - loss: 0.5684 - accuracy: 0.7221 - 2s/epoch - 110ms/step
Epoch 8/30
19/19 - 2s - loss: 0.4971 - accuracy: 0.7854 - 2s/epoch - 109ms/step
Epoch 9/30
19/19 - 2s - loss: 0.4204 - accuracy: 0.8153 - 2s/epoch - 107ms/step
Epoch 10/30
19/19 - 2s - loss: 0.4435 - accuracy: 0.7820 - 2s/epoch - 108ms/step
Epoch 11/30
19/19 - 2s - loss: 0.4415 - accuracy: 0.7987 - 2s/epoch - 109ms/step
Epoch 12/30
19/19 - 2s - loss: 0.4126 - accuracy: 0.8319 - 2s/epoch - 109ms/step
Epoch 13/30
19/19 - 2s - loss: 0.41

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 7s - loss: 0.6935 - accuracy: 0.4983 - 7s/epoch - 347ms/step
Epoch 2/30
19/19 - 1s - loss: 0.6903 - accuracy: 0.5648 - 929ms/epoch - 49ms/step
Epoch 3/30
19/19 - 1s - loss: 0.6338 - accuracy: 0.6611 - 919ms/epoch - 48ms/step
Epoch 4/30
19/19 - 1s - loss: 0.5168 - accuracy: 0.7558 - 919ms/epoch - 48ms/step
Epoch 5/30
19/19 - 1s - loss: 0.4602 - accuracy: 0.7973 - 934ms/epoch - 49ms/step
Epoch 6/30
19/19 - 1s - loss: 0.4578 - accuracy: 0.8156 - 942ms/epoch - 50ms/step
Epoch 7/30
19/19 - 1s - loss: 0.4380 - accuracy: 0.8223 - 963ms/epoch - 51ms/step
Epoch 8/30
19/19 - 1s - loss: 0.4203 - accuracy: 0.8339 - 934ms/epoch - 49ms/step
Epoch 9/30
19/19 - 1s - loss: 0.4022 - accuracy: 0.8455 - 923ms/epoch - 49ms/step
Epoch 10/30
19/19 - 1s - loss: 0.4551 - accuracy: 0.7807 - 972ms/epoch - 51ms/step
Epoch 11/30
19/19 - 1s - loss: 0.4701 - accuracy: 0.7641 - 927ms/epoch - 49ms/step
Epoch 12/30
19/19 - 1s - loss: 0.4389 - accuracy: 0.8156 - 943ms/epoch - 50ms/step
Epoch 13/30
19/

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30


In [ ]:
results_df = pd.DataFrame(grid_result.cv_results_)
results_df[['params', 'mean_test_score', 'rank_test_score']]
# Print the best parameters and corresponding accuracy

In [ ]:
print("\n",grid_result)
print("\n")
print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)


In [ ]:
best_model = grid_result.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)

from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)